In [18]:
import os

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from common import ModelManager


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten: nn.Flatten = nn.Flatten()
        self.linear_relu_stack: nn.Sequential = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logistics = self.linear_relu_stack(x)
        return logistics


In [19]:
def run_training_model():
    model: NeuralNetwork = NeuralNetwork()
    model_manager: ModelManager = ModelManager(
        model=NeuralNetwork(),
        loss_fn=nn.CrossEntropyLoss(),
        optimizer=optim.SGD(model.parameters(), lr=1e-3),
    )
    epochs = 5
    for t in range(epochs):
        print(f"Epoch {t + 1}\n-------------------------------")
        model_manager.train(data_loader=train_data_loader)
        model_manager.test(data_loader=test_data_loader)
    model_manager.save_parameters(model_params_path)


def run_loading_model():
    model_manager: ModelManager = ModelManager(
        model=NeuralNetwork(),
        loss_fn=nn.CrossEntropyLoss(),
        optimizer=optim.SGD(self.model.parameters(), lr=1e-3),
    )
    model_manager.load_parameters(model_params_path)
    model_manager.test(data_loader=test_data_loader)

In [22]:
if __name__ == '__main__':
    data_path: str = os.path.join("resources", "datasets", "fashion-mnist")
    model_params_path: str = os.path.join("resources", "models", "nn-fashion-mnist.pth")
    os.makedirs(data_path, exist_ok=True)
    os.makedirs(os.path.dirname(model_params_path), exist_ok=True)

    training_data: datasets.FashionMNIST = datasets.FashionMNIST(
        root=data_path,
        train=True,
        download=True,
        transform=ToTensor(),
    )

    test_data: datasets.FashionMNIST = datasets.FashionMNIST(
        root=data_path,
        train=False,
        download=True,
        transform=ToTensor(),
    )

    train_data_loader = DataLoader(training_data, batch_size=64)
    test_data_loader = DataLoader(test_data, batch_size=64)

    for X, y in test_data_loader:
        print(f"Shape of X [N, C, H, W] {X.shape}")
        print(f"Shape of y {y.shape}, {y.dtype}")
        break

    run_training_model()
    run_loading_model()

Shape of X [N, C, H, W] torch.Size([64, 1, 28, 28])
Shape of y torch.Size([64]), torch.int64


NameError: name 'torch' is not defined